In [ ]:
%pip install neo4j_graphrag langchain_openai torch

In [1]:
import os

os.environ['OPENAI_API_KEY'] = "sk-YOUR_KEY"
os.environ["NEO4J_URI"] = "YOUR_URI"
os.environ["NEO4J_USERNAME"] = "neo4j" 
os.environ["NEO4J_PASSWORD"] = "YOUR_N4J_KEY"

# 4. Queries

In [2]:
import textwrap
from langchain_community.graphs import Neo4jGraph
from neo4j import GraphDatabase
from neo4j_graphrag.retrievers import VectorRetriever
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.generation import GraphRAG
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings

import textwrap

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_USERNAME = os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")
NEO4J_DATABASE = 'neo4j'

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

kg = Neo4jGraph(url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE)


In [3]:
LISTA_PERGUNTAS = [
"Quais são os candidatos a eleição?",
"Qual o partido do candidato Requião?",
"Qual o partido de cada um dos candidatos?",
"Quem são os Vices?",
"Existem propostas específicas para o bairro Centro?",
"Quais as propostas para a área da saúde?",
"Quais são as propostas para a saúde do candidato Roberto Requião?",
"Temos algum candidato que não inclui em seu plano o tema 'Políticas Afirmativas e Direitos Humanos'",
"O Plano do candidato Luciano Ducci contempla quais tópicos?",
"Como o candidato Luizão Goulart enxerga a questão da saúde pública?",
"Como a candidata Cristina Graeml vê a questão da obrigatoriedade do plano de vacinação infantil?",
"Que propostas o candidato Roberto Requião traz para o ensino fundamental?",
"Qual candidato possui mais propostas na área da saúde?",
"Cada candidato tem uma bandeira, tema no qual foca maior quantidade de propostas. Quais são as principais bandeiras de cada candidato?",
"Qual dos candidatos a prefeito tem o maior número de propostas?",
"Qual candidato tem mais propostas na área da segurança pública?",
"Em que áreas as propostas de Ducci e de Ney Leprevost mais se assemelham e mais se diferenciam?",
"Qual candidato possui mais propostas para o transporte coletivo?",
"Quais os candidatos que mais se assemelham?",
"Meu candidato, o candidao Professor Luizão, não foi eleito para o segundo turno, estre os candidatos Cristina Graeml e Eduardo Pimentel, que passaram para um segundo turno, quais têm propostas que melhor se assemelham às de meu candidato, por área de gestão (Saúde, Educação, etc)?"
]

In [41]:
def avaliacao(perguntas, Q, **kwargs):
    i=1
    for pergunta in perguntas:
        print("PERGUNTA " + str(i) + ":", textwrap.fill(pergunta, 120))
        resposta = Q(pergunta)
        f_resposta = ""
        for key in list(resposta.keys()):
            f_resposta = f_resposta + key + ": " + resposta[key] + "\n"
        print(text_formater(f_resposta))
        print(70*"-")
        i=i+1

def text_formater(text):
    paragraphs = text.splitlines()
    wrapped_paragraphs = [textwrap.fill(paragraph, width=120) for paragraph in paragraphs]
    return "\n".join(wrapped_paragraphs)

## 4.1 RAG Q1

In [5]:
AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)

INDEX_NAME = "ponto_vidx"

# Connect to Neo4j database
driver = GraphDatabase.driver(NEO4J_URI, auth=AUTH)

# 2. Retriever
# Create Embedder object, needed to convert the user question (text) to a vector
embedder = OpenAIEmbeddings(model="text-embedding-3-large")

# Initialize the retriever
retrieverQ1 = VectorRetriever(driver, INDEX_NAME, embedder)

# 3. LLM
# Note: the OPENAI_API_KEY must be in the env vars
llm = OpenAILLM(model_name="gpt-4o", model_params={"temperature": 0})

# Initialize the RAG pipeline
rag = GraphRAG(retriever=retrieverQ1, llm=llm)

def colect_context(items):
    context = ""
    for item in items:
        context = context + item.content + "\n"
    return context


def Q1(question):
    response = rag.search(query_text=question, return_context=True)
    return {"CONTEXTO": colect_context(response.retriever_result.items), "RESPOSTA": response.answer}

In [53]:
avaliacao(LISTA_PERGUNTAS, Q1)

PERGUNTA 1: Quais são os candidatos a eleição?
CONTEXTO: {'texto': 'A candidatura à prefeitura do PSOL e da REDE carrega consigo a coragem e compromisso para combater
efetivamente o racismo em Curitiba.', 'candidato': 'ANDREA DO ROCIO CALDAS', 'tipo': 'proposta', 'nome': 'Compromisso
para combater o racismo em Curitiba.', 'textEmbedding': None}
{'texto': 'Luizão Goulart, candidato a Prefeito de Curitiba apresenta à sociedade curitibana e aos paranaenses em geral
a sua proposta de governo para o mandato de 2025/2028.', 'candidato': 'LUIZ GOULARTE ALVES', 'tipo': 'proposta', 'nome':
'Proposta de governo para o mandato de 2025/2028.', 'textEmbedding': None}
{'texto': 'A Federação PSOL – REDE está nestas eleições para propor uma Curitiba diferente.', 'candidato': 'ANDREA DO
ROCIO CALDAS', 'tipo': 'proposta', 'nome': 'Propor uma Curitiba diferente.', 'textEmbedding': None}
{'texto': 'A Coligação Curitiba Pode Mais apresenta uma visão clara para a cidade: liderar com integridade, inovação e


## 4.2 Cypher query: consulta à base usando liguagem cypher

In [56]:
# P1: Quais são os candidatos a eleição?
kg.query("MATCH (c:candidato) RETURN c.nome AS Candidato ORDER BY Candidato")

[{'Candidato': 'ANDREA DO ROCIO CALDAS'},
 {'Candidato': 'CRISTINA REIS GRAEML'},
 {'Candidato': 'EDUARDO PIMENTEL SLAVIERO'},
 {'Candidato': 'FELIPE GUSTAVO BOMBARDELLI'},
 {'Candidato': 'LUCIANO DUCCI'},
 {'Candidato': 'LUIZ GOULARTE ALVES'},
 {'Candidato': 'MARIA VICTORIA BORGHETTI BARROS'},
 {'Candidato': 'NEY LEPREVOST NETO'},
 {'Candidato': 'ROBERTO REQUIÃO DE MELLO E SILVA'},
 {'Candidato': 'SAMUEL DE MATTOS FIGUEIREDO'}]

In [ ]:
# P2 Qual o partido do candidato Requião?
kg.query("MATCH (c:candidato {nome: 'ROBERTO REQUIÃO DE MELLO E SILVA'}) RETURN c.partido AS Partido")

NameError: name 'kg' is not defined

In [61]:
# P3: Qual o partido de cada um dos candidatos?
kg.query("MATCH (c:candidato) RETURN c.nome AS Candidato, c.partido AS Partido ORDER BY Candidato")

[{'Candidato': 'ANDREA DO ROCIO CALDAS', 'Partido': 'FEDERAÇÃO PSOL - REDE'},
 {'Candidato': 'CRISTINA REIS GRAEML', 'Partido': 'PMB'},
 {'Candidato': 'EDUARDO PIMENTEL SLAVIERO',
  'Partido': 'Curitiba Amor e Inovação'},
 {'Candidato': 'FELIPE GUSTAVO BOMBARDELLI', 'Partido': 'PCO'},
 {'Candidato': 'LUCIANO DUCCI', 'Partido': 'Curitiba + Social e Humana'},
 {'Candidato': 'LUIZ GOULARTE ALVES', 'Partido': 'SOLIDARIEDADE'},
 {'Candidato': 'MARIA VICTORIA BORGHETTI BARROS',
  'Partido': 'CURITIBA MELHOR PARA TODOS'},
 {'Candidato': 'NEY LEPREVOST NETO', 'Partido': 'CURITIBA PODE MAIS'},
 {'Candidato': 'ROBERTO REQUIÃO DE MELLO E SILVA', 'Partido': 'MOBILIZA'},
 {'Candidato': 'SAMUEL DE MATTOS FIGUEIREDO', 'Partido': 'PSTU'}]

In [62]:
# P4: Quem são os Vices?
kg.query("MATCH (c:candidato) RETURN c.nome AS Candidato, c.vice AS Vice ORDER BY Candidato")

[{'Candidato': 'ANDREA DO ROCIO CALDAS', 'Vice': 'LETICIA FARIA'},
 {'Candidato': 'CRISTINA REIS GRAEML', 'Vice': 'Dr Jairo Filho'},
 {'Candidato': 'EDUARDO PIMENTEL SLAVIERO', 'Vice': 'Paulo Martins'},
 {'Candidato': 'FELIPE GUSTAVO BOMBARDELLI', 'Vice': 'Paulo Costycha'},
 {'Candidato': 'LUCIANO DUCCI', 'Vice': 'Goura'},
 {'Candidato': 'LUIZ GOULARTE ALVES', 'Vice': 'Tiago Chico'},
 {'Candidato': 'MARIA VICTORIA BORGHETTI BARROS',
  'Vice': 'Walter Petruzziello'},
 {'Candidato': 'NEY LEPREVOST NETO', 'Vice': 'Rosângela Moro'},
 {'Candidato': 'ROBERTO REQUIÃO DE MELLO E SILVA', 'Vice': 'Marcelo Henrique'},
 {'Candidato': 'SAMUEL DE MATTOS FIGUEIREDO', 'Vice': 'Leo Martinez'}]

In [78]:
# P5: Existem propostas específicas para o bairro Centro?
cypher = f"""
WITH genai.vector.encode('bairro Centro', 'OpenAI', {{token:'{OPENAI_API_KEY}', model:'text-embedding-3-large'}}) AS query_vector
CALL db.index.vector.queryNodes('ponto_vidx', 5000, query_vector) YIELD node, score WHERE score > 0.75 
RETURN node.nome, node.texto, score ORDER BY score DESC
"""
kg.query(cypher)

[{'node.nome': 'Centro Histórico - Praça Garibaldi.',
  'node.texto': 'Centro Histórico - Praça Garibaldi.',
  'score': 0.7771568298339844},
 {'node.nome': 'Necessidade de revitalização do Centro.',
  'node.texto': 'O Centro necessita de revitalização para recuperar sua vitalidade econômica e social, além de proporcionar melhores condições de vida para os residentes e visitantes.',
  'score': 0.7566566467285156},
 {'node.nome': 'Intervenções para reativar o movimento na região central.',
  'node.texto': 'A reativação do Centro será consolidada com um grande elenco de intervenções de iniciativa da Prefeitura, em parceria com entidades de classe e a sociedade organizada, para reativar o movimento na região central com ocupação diversificada.',
  'score': 0.7548480033874512},
 {'node.nome': 'Terminal Boa Vista',
  'node.texto': 'Terminal localizado no bairro Boa Vista, Regional Boa Vista.',
  'score': 0.7506136894226074}]

In [79]:
# P6: Quais as propostas para a área da saúde?
kg.query("MATCH (e:eixo {STD_Header: 'Saúde'})-[:propoe]->(p:ponto) RETURN p.nome AS Proposta, p.texto AS Descrição ORDER BY p.nome")

[{'Proposta': 'Acabar com a FEAS.',
  'Descrição': 'Por isso, propomos: acabar com a FEAS, chega de terceirização.'},
 {'Proposta': 'Acabar com o capitalismo.',
  'Descrição': 'A saída não é a fé, não é acreditar. É tratamento, é discussão de sociedade, e com toda certeza é acabar com o capitalismo.'},
 {'Proposta': 'Acesso Universal e Atendimento Humanizado.',
  'Descrição': 'Garantir que todos os cidadãos de Curitiba tenham acesso facilitado a serviços de saúde de qualidade, respeitando os princípios de equidade e individualidade.'},
 {'Proposta': 'Acesso a orientações sobre planejamento familiar.',
  'Descrição': 'Garantir que as famílias, especialmente as mulheres, tenham acesso a orientações sobre planejamento familiar e métodos contraceptivos que estão disponíveis no SUS.'},
 {'Proposta': 'Acesso ao planejamento familiar para mulheres.',
  'Descrição': 'O planejamento familiar e reprodutivo é um direito das mulheres! E é nosso dever, enquanto governo municipal, garantir que todas

In [80]:
# P7: Quais são as propostas para a saúde do candidato Roberto Requião?
kg.query("MATCH (c:candidato {nome: 'ROBERTO REQUIÃO DE MELLO E SILVA'})-[:planeja]->(:plano)-[:contem]->(e:eixo {STD_Header: 'Saúde'})-[:propoe]->(p:ponto) RETURN p.nome AS Proposta, p.texto AS Descrição ORDER BY Proposta")

[{'Proposta': 'Aumento da capacidade de atendimento nos postos de saúde.',
  'Descrição': 'Aumentar a capacidade de atendimento nos postos de saúde para reduzir o tempo de espera e melhorar a qualidade do serviço, além de investir na aquisição e revisão periódica de equipamentos médicos de última geração para diagnósticos mais precisos e tratamentos mais eficazes.'},
 {'Proposta': 'Aumento do número de postos de saúde e construção de novos hospitais.',
  'Descrição': 'Aumentar número de postos de saúde nos bairros e construir novos hospitais.'},
 {'Proposta': 'Criação de um cadastro único de profissionais da saúde.',
  'Descrição': 'Criar um cadastro único de profissionais da saúde para serem contratados em emergências epidemiológicas.'},
 {'Proposta': 'Desenvolvimento de campanhas de saúde preventiva.',
  'Descrição': 'Desenvolver campanhas de prevenção e educação em saúde para a população, abordando temas como vacinação, higiene, nutrição e atividade física.'},
 {'Proposta': 'Elimina

In [81]:
# P8: Temos algum candidato que não inclui em seu plano o tema 'Políticas Afirmativas e Direitos Humanos'
kg.query("MATCH (c:candidato) WHERE NOT (c)-[:planeja]->(:plano)-[:contem]->(:eixo {STD_Header: 'Políticas Afirmativas e Direitos Humanos'})-[:propoe]->(:ponto) RETURN c.nome AS Candidato, c.partido AS Partido ORDER BY Candidato")


[{'Candidato': 'CRISTINA REIS GRAEML', 'Partido': 'PMB'},
 {'Candidato': 'FELIPE GUSTAVO BOMBARDELLI', 'Partido': 'PCO'},
 {'Candidato': 'LUCIANO DUCCI', 'Partido': 'Curitiba + Social e Humana'}]

In [82]:
# P9: O Plano do candidato Luciano Ducci contempla quais tópicos?
kg.query("MATCH (c:candidato)-[]-(:plano)-[]-(e:eixo) WHERE c.nome='LUCIANO DUCCI' RETURN e.Header_1")


[{'e.Header_1': 'Introdução'},
 {'e.Header_1': 'Educação'},
 {'e.Header_1': 'Saúde'},
 {'e.Header_1': 'Segurança Pública'},
 {'e.Header_1': 'Ambiente e Prevenção ao Aquecimento Global'},
 {'e.Header_1': 'Tratamento dos Resíduos da Atividade Humana na Cidade'},
 {'e.Header_1': 'Transporte Público Urbano e Mobilidade'},
 {'e.Header_1': 'Infraestrutura Urbana'},
 {'e.Header_1': 'Habitação de Interesse Social e Regularização Fundiária'},
 {'e.Header_1': 'Conservação da cidade'},
 {'e.Header_1': 'Trabalho, Empreendedorismo e Renda'},
 {'e.Header_1': 'Propostas para o Turismo'},
 {'e.Header_1': 'Propostas para o Artesanato'},
 {'e.Header_1': 'Segurança Alimentar e Nutricional'},
 {'e.Header_1': 'Rede Comercial de Abastecimento'},
 {'e.Header_1': 'Assistência Social e Cidadania'},
 {'e.Header_1': 'Ampliar a Participação Popular'},
 {'e.Header_1': 'Atividade Física, Esporte e Lazer'},
 {'e.Header_1': 'Cultura'}]

In [20]:
 # P10: Como o candidato Luizão Goulart enxerga a questão da saúde pública?

CYPHER = f"""
WITH genai.vector.encode('saúde pública', 'OpenAI', {{token:'{OPENAI_API_KEY}', model:'text-embedding-3-large'}}) AS query_vector
CALL db.index.vector.queryNodes('ponto_vidx', 5000, query_vector) YIELD node, score
WHERE score > 0.70
MATCH (c:candidato {{nome: 'LUIZ GOULARTE ALVES'}})-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(node)
RETURN node.nome AS Proposta, node.texto AS Descrição, score ORDER BY  score DESC
"""
kg.query(CYPHER)

[{'Proposta': 'Promover a prevenção e educação em saúde.',
  'Descrição': 'Promoverá a prevenção, a educação em saúde e a conscientização sobre hábitos saudáveis.',
  'score': 0.7509140968322754},
 {'Proposta': 'Criar programas de exercício ao ar livre.',
  'Descrição': 'Criar programas de exercício ao ar livre articulados às políticas de saúde e assistência social.',
  'score': 0.7330794334411621},
 {'Proposta': 'Retirada de carcaça animal.',
  'Descrição': 'Ação efetiva na retirada de carcaça animal para evitar riscos à saúde pública.',
  'score': 0.7271952629089355},
 {'Proposta': 'Uso da alimentação saudável como medida preventiva.',
  'Descrição': 'Potencializar o uso da alimentação saudável, como medida preventiva contra doenças - articulando com as crianças nas escolas.',
  'score': 0.7236886024475098},
 {'Proposta': 'Popularizar a telemedicina.',
  'Descrição': 'Como modernização aumentar e popularizar a telemedicina que veio para ficar, articulando com o programa Saúde Já.',
 

In [21]:
# P11: Como a candidata Cristina Graeml vê a questão da obrigatoriedade do plano de vacinação infantil?
CYPHER = f"""
WITH genai.vector.encode('obrigatoriedade do plano de vacinação infantil', 'OpenAI', {{token:'{OPENAI_API_KEY}', model:'text-embedding-3-large'}}) AS query_vector
CALL db.index.vector.queryNodes('ponto_vidx', 5000, query_vector) YIELD node, score
WHERE score > 0.70
MATCH (c:candidato {{nome: 'CRISTINA REIS GRAEML'}})-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(node)
RETURN node.nome AS Proposta, node.texto AS Descrição, score ORDER BY  score DESC
"""
kg.query(CYPHER)

[{'Proposta': 'Direito à Vida Desde a Concepção.',
  'Descrição': 'Políticas de saúde e sociais que protejam tanto as mães quanto os filhos desde a sua concepção serão priorizadas, demonstrando o valor da vida e o renovo da esperança. A defesa da vida estará no centro de nossas ações.',
  'score': 0.7194528579711914},
 {'Proposta': 'Desenvolvimento de medidas para eliminação do número de mães e crianças que aguardam vagas nos CMEIs.',
  'Descrição': 'Desenvolvimento de medidas para eliminação do número de mães e crianças que aguardam vagas nos CMEIs.',
  'score': 0.7193212509155273},
 {'Proposta': 'Compromisso com a criação de um sistema de saúde acessível.',
  'Descrição': 'Este plano de saúde pública para Curitiba reflete nosso compromisso com a criação de um sistema de saúde acessível, eficiente e humanizado.',
  'score': 0.7147412300109863},
 {'Proposta': 'Programa Saúde na Escola.',
  'Descrição': 'Integrar as escolas com a rede de saúde para ações preventivas e educativas, promov

In [22]:
# P12: Que propostas o candidato Roberto Requião traz para o ensino fundamental?
CYPHER = f"""
WITH genai.vector.encode('ensino fundamental', 'OpenAI', {{token:'{OPENAI_API_KEY}', model:'text-embedding-3-large'}}) AS query_vector
CALL db.index.vector.queryNodes('ponto_vidx', 5000, query_vector) YIELD node, score
WHERE score > 0.70
MATCH (c:candidato {{nome: 'ROBERTO REQUIÃO DE MELLO E SILVA'}})-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(node)
RETURN node.nome AS Proposta, node.texto AS Descrição, score ORDER BY  score DESC
"""
kg.query(CYPHER)

[{'Proposta': 'Educação Inclusiva',
  'Descrição': 'Desenvolver ações que reduzam a evasão escolar, a defasagem idade-série e combatam o analfabetismo na educação municipal em todos os níveis e modalidades. Fortalecer a formação de professores e profissionais da educação nas áreas relacionadas ao ensino de história e cultura afro-brasileira, africana e indígena, promovendo uma educação inclusiva.',
  'score': 0.7301244735717773},
 {'Proposta': 'Escola integral.',
  'Descrição': 'Escola integral, para garantir que as crianças possam estar acompanhadas de profissionais durante todo o dia, permitindo que os pais possam trabalhar tranquilos, além de garantir uma alimentação equilibrada.',
  'score': 0.7256898880004883},
 {'Proposta': 'Educação inclusiva, equitativa e de qualidade.',
  'Descrição': 'Assegurar uma educação inclusiva, equitativa e de qualidade, promovendo oportunidades de aprendizagem ao longo da vida para todos.',
  'score': 0.7027716636657715}]

In [23]:
# P13: Qual candidato possui mais propostas na área da saúde?
CYPHER = """
MATCH (c:candidato)-[:planeja]->(:plano)-[:contem]->(e:eixo)-[:propoe]->(p:ponto) WHERE e.STD_Header = 'Saúde'
WITH c.nome AS Candidato, COUNT(p) AS Numero_de_propostas RETURN Candidato, Numero_de_propostas ORDER BY
Numero_de_propostas DESC LIMIT 1
"""
kg.query(CYPHER)

[{'Candidato': 'SAMUEL DE MATTOS FIGUEIREDO', 'Numero_de_propostas': 68}]

In [25]:
# P14: Cada candidato tem uma bandeira, tema no qual foca maior quantidade de propostas. Quais são as principais bandeiras de cada candidato?

CYPHER = """
MATCH (c:candidato)-[:planeja]->(:plano)-[:contem]->(e:eixo)-[:propoe]->(p:ponto)
WITH c.nome AS candidato, e.STD_Header AS header, COUNT(p) AS total_propostas
WITH candidato, header, total_propostas
ORDER BY candidato, total_propostas DESC
WITH candidato, COLLECT({header: header, total: total_propostas}) AS header_data
RETURN candidato, header_data[0].header AS maior_header
"""

kg.query(CYPHER)

[{'candidato': 'ANDREA DO ROCIO CALDAS',
  'maior_header': 'Políticas Afirmativas e Direitos Humanos'},
 {'candidato': 'CRISTINA REIS GRAEML',
  'maior_header': 'Desenvolvimento Econômico, Emprego, Renda e Tecnologia'},
 {'candidato': 'EDUARDO PIMENTEL SLAVIERO', 'maior_header': 'Urbanismo'},
 {'candidato': 'FELIPE GUSTAVO BOMBARDELLI', 'maior_header': 'Introdução'},
 {'candidato': 'LUCIANO DUCCI',
  'maior_header': 'Cultura, Esporte, Lazer e Turismo'},
 {'candidato': 'LUIZ GOULARTE ALVES',
  'maior_header': 'Políticas Afirmativas e Direitos Humanos'},
 {'candidato': 'MARIA VICTORIA BORGHETTI BARROS', 'maior_header': 'Educação'},
 {'candidato': 'NEY LEPREVOST NETO', 'maior_header': 'Gestão e Transparência'},
 {'candidato': 'ROBERTO REQUIÃO DE MELLO E SILVA',
  'maior_header': 'Políticas Afirmativas e Direitos Humanos'},
 {'candidato': 'SAMUEL DE MATTOS FIGUEIREDO', 'maior_header': 'Saúde'}]

In [26]:
# P15: Qual dos candidatos a prefeito tem o maior número de propostas?
CYPHER = """
MATCH (c:candidato)-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(p:ponto)
WITH c.nome AS Candidato, COUNT(p) AS Numero_de_propostas
RETURN Candidato, Numero_de_propostas
ORDER BY Numero_de_propostas DESC
LIMIT 1
"""

kg.query(CYPHER)

[{'Candidato': 'EDUARDO PIMENTEL SLAVIERO', 'Numero_de_propostas': 778}]

In [57]:
# P16: Qual candidato tem mais propostas na área da segurança pública?
CYPHER ="""
MATCH (c:candidato)-[:planeja]->(:plano)-[:contem]->(e:eixo {STD_Header: 'Segurança Pública'})-[:propoe]->(p:ponto)
WITH c.nome AS Candidato, COUNT(p) AS Numero_de_propostas
RETURN Candidato, Numero_de_propostas
ORDER BY Numero_de_propostas DESC
"""
kg.query(CYPHER)

[{'Candidato': 'CRISTINA REIS GRAEML', 'Numero_de_propostas': 43},
 {'Candidato': 'SAMUEL DE MATTOS FIGUEIREDO', 'Numero_de_propostas': 27},
 {'Candidato': 'ANDREA DO ROCIO CALDAS', 'Numero_de_propostas': 17},
 {'Candidato': 'LUIZ GOULARTE ALVES', 'Numero_de_propostas': 17},
 {'Candidato': 'NEY LEPREVOST NETO', 'Numero_de_propostas': 15},
 {'Candidato': 'LUCIANO DUCCI', 'Numero_de_propostas': 15},
 {'Candidato': 'MARIA VICTORIA BORGHETTI BARROS', 'Numero_de_propostas': 12},
 {'Candidato': 'ROBERTO REQUIÃO DE MELLO E SILVA', 'Numero_de_propostas': 8},
 {'Candidato': 'EDUARDO PIMENTEL SLAVIERO', 'Numero_de_propostas': 7}]

In [84]:
# P17: Em que áreas as propostas de Ducci e de Ney Leprevost mais se assemelham e mais se diferenciam?
kg.query("""
MATCH (c1:candidato)-[:planeja]->(:plano)-[:contem]->(e1:eixo)-[:propoe]->(p1:ponto)-[s:similar_to]-(p2:ponto)<-[:propoe]-(e2:eixo)<-[:contem]-(:plano)<-[:planeja]-(c2:candidato)
WHERE c1.nome = 'LUCIANO DUCCI' AND c2.nome = 'NEY LEPREVOST NETO'
WITH e1.STD_Header AS header, s.cos_similarity AS similarity
RETURN header, AVG(similarity) AS avg_cos_similarity
ORDER BY avg_cos_similarity DESC
""")

[{'header': 'Habitação e Ocupação Urbana',
  'avg_cos_similarity': 0.6038254842773098},
 {'header': 'Gestão e Transparência',
  'avg_cos_similarity': 0.5905440349150055},
 {'header': 'Assistência e Ação Social',
  'avg_cos_similarity': 0.5886045102076987},
 {'header': 'Urbanismo', 'avg_cos_similarity': 0.5694888610794195},
 {'header': 'Transporte Público e Mobilidade',
  'avg_cos_similarity': 0.5626330986383449},
 {'header': 'Educação', 'avg_cos_similarity': 0.5550218259063885},
 {'header': 'Meio Ambiente e crise climática',
  'avg_cos_similarity': 0.5449334575087671},
 {'header': 'Desenvolvimento Econômico, Emprego, Renda e Tecnologia',
  'avg_cos_similarity': 0.5430718645148556},
 {'header': 'Segurança Pública', 'avg_cos_similarity': 0.5421918271489286},
 {'header': 'Introdução', 'avg_cos_similarity': 0.5418651448477234},
 {'header': 'Cultura, Esporte, Lazer e Turismo',
  'avg_cos_similarity': 0.5380073112727283},
 {'header': 'Zeladoria e Serviços', 'avg_cos_similarity': 0.5327886018

In [89]:
# P18: Qual candidato possui mais propostas para o transporte coletivo?
CYPHER = f"""
WITH genai.vector.encode('Transporte Coletivo', 'OpenAI', {{token:'{OPENAI_API_KEY}', model:'text-embedding-3-large'}}) AS query_vector
CALL db.index.vector.queryNodes('ponto_vidx', 5000, query_vector) YIELD node, score
WHERE score > 0.75 
MATCH (c:candidato)-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(node)
RETURN c.nome AS Candidato, COUNT(node) AS Numero_de_pontos
ORDER BY Numero_de_pontos DESC
"""

kg.query(CYPHER)

[{'Candidato': 'EDUARDO PIMENTEL SLAVIERO', 'Numero_de_pontos': 11},
 {'Candidato': 'MARIA VICTORIA BORGHETTI BARROS', 'Numero_de_pontos': 9},
 {'Candidato': 'ANDREA DO ROCIO CALDAS', 'Numero_de_pontos': 8},
 {'Candidato': 'CRISTINA REIS GRAEML', 'Numero_de_pontos': 7},
 {'Candidato': 'LUCIANO DUCCI', 'Numero_de_pontos': 6},
 {'Candidato': 'SAMUEL DE MATTOS FIGUEIREDO', 'Numero_de_pontos': 5},
 {'Candidato': 'LUIZ GOULARTE ALVES', 'Numero_de_pontos': 4},
 {'Candidato': 'NEY LEPREVOST NETO', 'Numero_de_pontos': 3},
 {'Candidato': 'ROBERTO REQUIÃO DE MELLO E SILVA', 'Numero_de_pontos': 2},
 {'Candidato': 'FELIPE GUSTAVO BOMBARDELLI', 'Numero_de_pontos': 1}]

In [92]:
# P19: Quais os candidatos que mais se assemelham?

CYPHER = """
MATCH (c1:candidato)-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(p1:ponto)-[sim:similar_to]->(p2:ponto)<-[:propoe]-(:eixo)<-[:contem]-(:plano)<-[:planeja]-(c2:candidato)
WHERE c1 <> c2
WITH c1.nome AS Candidato1, c2.nome AS Candidato2, AVG(sim.cos_similarity) AS Similarity
RETURN Candidato1, Candidato2, Similarity
ORDER BY Similarity DESC
"""
kg.query(CYPHER)

[{'Candidato1': 'ROBERTO REQUIÃO DE MELLO E SILVA',
  'Candidato2': 'CRISTINA REIS GRAEML',
  'Similarity': 0.6594601074205814},
 {'Candidato1': 'ROBERTO REQUIÃO DE MELLO E SILVA',
  'Candidato2': 'LUIZ GOULARTE ALVES',
  'Similarity': 0.6523129001804846},
 {'Candidato1': 'ROBERTO REQUIÃO DE MELLO E SILVA',
  'Candidato2': 'ANDREA DO ROCIO CALDAS',
  'Similarity': 0.646511527925379},
 {'Candidato1': 'LUIZ GOULARTE ALVES',
  'Candidato2': 'ANDREA DO ROCIO CALDAS',
  'Similarity': 0.6405277656766248},
 {'Candidato1': 'LUIZ GOULARTE ALVES',
  'Candidato2': 'CRISTINA REIS GRAEML',
  'Similarity': 0.6374964274409145},
 {'Candidato1': 'ROBERTO REQUIÃO DE MELLO E SILVA',
  'Candidato2': 'EDUARDO PIMENTEL SLAVIERO',
  'Similarity': 0.630221737941784},
 {'Candidato1': 'LUIZ GOULARTE ALVES',
  'Candidato2': 'EDUARDO PIMENTEL SLAVIERO',
  'Similarity': 0.630092696426824},
 {'Candidato1': 'CRISTINA REIS GRAEML',
  'Candidato2': 'LUIZ GOULARTE ALVES',
  'Similarity': 0.6293506193874367},
 {'Candida

In [95]:
# P20: Meu candidato, o candidao Professor Luizão, não foi eleito para o segundo turno, estre os candidatos Cristina Graeml e
# Eduardo Pimentel, que passaram para um segundo turno, quais têm propostas que melhor se assemelham às de meu candidato,
# por área de gestão (Saúde, Educação, etc)?

EDU = kg.query("""
MATCH (c1:candidato)-[:planeja]->(:plano)-[:contem]->(e1:eixo)-[:propoe]->(:ponto)-[s:similar_to]-(:ponto)<-[:propoe]-(:eixo)<-[:contem]-(:plano)<-[:planeja]-(c2:candidato)
WHERE c1.nome = 'LUIZ GOULARTE ALVES' AND c2.nome = 'EDUARDO PIMENTEL SLAVIERO'
WITH e1.STD_Header AS header, s.cos_similarity AS similarity
RETURN header, AVG(similarity)
""")
CRIS = kg.query("""
MATCH (c1:candidato)-[:planeja]->(:plano)-[:contem]->(e1:eixo)-[:propoe]->(:ponto)-[s:similar_to]-(:ponto)<-[:propoe]-(:eixo)<-[:contem]-(:plano)<-[:planeja]-(c2:candidato)
WHERE c1.nome = 'LUIZ GOULARTE ALVES' AND c2.nome = 'CRISTINA REIS GRAEML'
WITH e1.STD_Header AS header, s.cos_similarity AS similarity
RETURN header, AVG(similarity)
""")

for ed in EDU:
    for cr in CRIS:
        if ed['header'] == cr['header']:
            if ed['AVG(similarity)'] > cr['AVG(similarity)']:
                print(ed['header'], ":", 'EDUARDO PIMENTEL SLAVIERO')
            elif ed['AVG(similarity)'] < cr['AVG(similarity)']:
                print(ed['header'], ":", 'CRISTINA REIS GRAEML')
            else:
                print(ed['header'], ":", 'SCORE EQUIVALENTE')



Introdução : CRISTINA REIS GRAEML
Transporte Público e Mobilidade : CRISTINA REIS GRAEML
Urbanismo : CRISTINA REIS GRAEML
Meio Ambiente e crise climática : CRISTINA REIS GRAEML
Desenvolvimento Econômico, Emprego, Renda e Tecnologia : CRISTINA REIS GRAEML
Saúde : CRISTINA REIS GRAEML
Educação : EDUARDO PIMENTEL SLAVIERO
Segurança Alimentar : CRISTINA REIS GRAEML
Assistência e Ação Social : EDUARDO PIMENTEL SLAVIERO
Gestão e Transparência : CRISTINA REIS GRAEML
Políticas Afirmativas e Direitos Humanos : EDUARDO PIMENTEL SLAVIERO
Cultura, Esporte, Lazer e Turismo : CRISTINA REIS GRAEML
Habitação e Ocupação Urbana : EDUARDO PIMENTEL SLAVIERO
Segurança Pública : CRISTINA REIS GRAEML


## 4.3 Text2Cypher - Usa LLM para escrever o código cypher à partir da pergunta do usuário

In [87]:
from neo4j_graphrag.retrievers import Text2CypherRetriever

# (Optional) Specify your own Neo4j schema
kg.refresh_schema()
schema = kg.get_schema

# (Optional) Provide user input/query pairs for the LLM to use as examples
examples = [
"USER INPUT: 'Qual o nome dos candidatos a eleição?' QUERY: MATCH (c:candidato) RETURN c.nome AS Candidato ORDER BY Candidato",
 "USER INPUT: 'Qual o vice do candidato Ney Leprevost?' QUERY: MATCH (c:candidato {nome: 'NEY LEPREVOST NETO'}) RETURN c.vice AS Vice",
 "USER INPUT: 'Qual a coligação de cada um dos candidatos?' QUERY: MATCH (c:candidato) RETURN c.nome AS Candidato, c.coalizao AS Coligação ORDER BY Candidato",
 "USER INPUT: 'Quem são os Vices de cada candidato?' QUERY: MATCH (c:candidato) RETURN c.nome AS Candidato, c.vice AS Vice ORDER BY Candidato",
 "USER INPUT: 'Existem propostas específicas para o bairro Bacacheri?' QUERY: WITH genai.vector.encode('Bacacheri', 'OpenAI', {token:'sk-YOUR_KEY', model:'text-embedding-3-large'}) AS query_vector CALL db.index.vector.queryNodes('ponto_vidx', 5000, query_vector) YIELD node, score WHERE score > 0.75 RETURN node.nome, node.texto, score ORDER BY score DESC",
 "USER INPUT: 'Quais as propostas para a área da Educação?' QUERY: MATCH (e:eixo {STD_Header: 'Educação'})-[:propoe]->(p:ponto) RETURN p.nome AS Proposta, p.texto AS Descrição ORDER BY p.nome",
 "USER INPUT: 'Quais são as propostas para a saúde do candidato Maria Vitória?' QUERY: MATCH (c:candidato {nome: 'MARIA VICTORIA BORGHETTI BARROS'})-[:planeja]->(:plano)-[:contem]->(e:eixo {STD_Header: 'Saúde'})-[:propoe]->(p:ponto) RETURN p.nome AS Proposta, p.texto AS Descrição ORDER BY Proposta",
 "USER INPUT: 'Temos algum candidato que não inclui em seu plano o tema 'Zeladoria e Serviços'' QUERY: MATCH (c:candidato) WHERE NOT (c)-[:planeja]->(:plano)-[:contem]->(:eixo {STD_Header: 'Zeladoria e Serviços'})-[:propoe]->(:ponto) RETURN c.nome AS Candidato_que_não_contempla_o_tema ORDER BY Candidato_que_não_contempla_o_tema",
 "USER INPUT: 'O Plano do candidato Andrea Caldas contempla quais tópicos?' QUERY: MATCH (c:candidato)-[]-(:plano)-[]-(e:eixo) WHERE c.nome='ANDREA DO ROCIO CALDAS' RETURN e.Header_1",
 "USER INPUT: 'Como o candidato Luizão Goulart enxerga a questão da educação básica?' QUERY: WITH genai.vector.encode('educação básica', 'OpenAI', {token:'sk-YOUR_KEY', model:'text-embedding-3-large'}) AS query_vector CALL db.index.vector.queryNodes('ponto_vidx', 5000, query_vector) YIELD node, score WHERE score > 0.70 MATCH (c:candidato {nome: 'LUIZ GOULARTE ALVES'})-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(node) RETURN node.nome AS Proposta, node.texto AS Descrição, score ORDER BY  score DESC",
 "USER INPUT: 'Como a candidata Cristina Graeml vê a questão da diversidade e gênero?' QUERY: WITH genai.vector.encode('diversidade e gênero', 'OpenAI', {token:'sk-YOUR_KEY', model:'text-embedding-3-large'}) AS query_vector CALL db.index.vector.queryNodes('ponto_vidx', 5000, query_vector) YIELD node, score WHERE score > 0.70 MATCH (c:candidato {nome: 'CRISTINA REIS GRAEML'})-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(node) RETURN node.nome AS Proposta, node.texto AS Descrição, score ORDER BY  score DESC",
 "USER INPUT: 'Que propostas o candidato Roberto Requião traz para o desenvovlimento turístico?' QUERY: WITH genai.vector.encode('desenvovlimento turístico', 'OpenAI', {token:'sk-YOUR_KEY', model:'text-embedding-3-large'}) AS query_vector CALL db.index.vector.queryNodes('ponto_vidx', 5000, query_vector) YIELD node, score WHERE score > 0.70 MATCH (c:candidato {nome: 'ROBERTO REQUIÃO DE MELLO E SILVA'})-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(node) RETURN node.nome AS Proposta, node.texto AS Descrição, score ORDER BY  score DESC",
 "USER INPUT: 'Qual candidato possui mais propostas na área da Habitação e Ocupação Urbana ?' QUERY: MATCH (c:candidato)-[:planeja]->(:plano)-[:contem]->(e:eixo)-[:propoe]->(p:ponto) WHERE e.STD_Header = 'Habitação e Ocupação Urbana' WITH c.nome AS Candidato, COUNT(p) AS Numero_de_propostas_na_Saúde RETURN Candidato, Numero_de_propostas_na_Saúde ORDER BY Numero_de_propostas_na_Saúde DESC LIMIT 1",
 "USER INPUT: 'Cada candidato tem um tema no qual foca maior quantidade de propostas. Quais são estes temas para de cada candidato?' QUERY: MATCH (c:candidato)-[:planeja]->(:plano)-[:contem]->(e:eixo)-[:propoe]->(p:ponto) WITH c.nome AS candidato, e.STD_Header AS header, COUNT(p) AS total_propostas WITH candidato, header, total_propostas ORDER BY candidato, total_propostas DESC WITH candidato, COLLECT({header: header, total: total_propostas}) AS header_data RETURN candidato, header_data[0].header AS maior_header",
 "USER INPUT: 'Qual dos candidatos a prefeito tem o menor número de propostas?' QUERY: MATCH (c:candidato)-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(p:ponto) WITH c.nome AS Candidato, COUNT(p) AS Numero_de_propostas RETURN Candidato, Numero_de_propostas ORDER BY Numero_de_propostas LIMIT 1",
 "USER INPUT: 'Qual candidato tem mais propostas na área da Assistência e Ação Social?' QUERY: MATCH (c:candidato)-[:planeja]->(:plano)-[:contem]->(e:eixo {STD_Header: 'Assistência e Ação Social'})-[:propoe]->(p:ponto) WITH c.nome AS Candidato, COUNT(p) AS Numero_de_propostas_para_segurança_pública RETURN Candidato, Numero_de_propostas_para_segurança_pública ORDER BY Numero_de_propostas_para_segurança_pública DESC",
 "USER INPUT: 'Em que áreas as propostas de Requião e de Eduardo Pimentel mais se assemelham e mais se diferenciam?' QUERY: MATCH (c1:candidato)-[:planeja]->(:plano)-[:contem]->(e1:eixo)-[:propoe]->(p1:ponto)-[s:similar_to]-(p2:ponto)<-[:propoe]-(e2:eixo)<-[:contem]-(:plano)<-[:planeja]-(c2:candidato) WHERE c1.nome = 'ROBERTO REQUIÃO DE MELLO E SILVA' AND c2.nome = 'EDUARDO PIMENTEL SLAVIERO' WITH e1.STD_Header AS header, s.cos_similarity AS similarity RETURN header, AVG(similarity) AS avg_cos_similarity ORDER BY avg_cos_similarity DESC",
 "USER INPUT: 'Qual candidato possui mais propostas para o modal de transporte bicicleta?' QUERY: WITH genai.vector.encode('modal de transporte bicicleta', 'OpenAI', {token:'sk-YOUR_KEY', model:'text-embedding-3-large'}) AS query_vector CALL db.index.vector.queryNodes('ponto_vidx', 5000, query_vector) YIELD node, score WHERE score > 0.75 MATCH (c:candidato)-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(node) RETURN c.nome AS Candidato, COUNT(node) AS Numero_de_pontos ORDER BY Numero_de_pontos DESC",
 "USER INPUT: 'Quais os candidatos que mais se assemelham entre si?' QUERY: MATCH (c1:candidato)-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(p1:ponto)-[sim:similar_to]->(p2:ponto)<-[:propoe]-(:eixo)<-[:contem]-(:plano)<-[:planeja]-(c2:candidato) WHERE c1 <> c2 WITH c1.nome AS Candidato1, c2.nome AS Candidato2, AVG(sim.cos_similarity) AS Similarity RETURN Candidato1, Candidato2, Similarity ORDER BY Similarity DESC",
 "USER INPUT: 'Meu candidato, o candidao Requião, não foi eleito para o segundo turno, estre os candidatos Luciano Ducci e Professor Luizão, quais têm propostas que melhor se assemelham às de meu candidato, por área de gestão (Saúde, Educação, etc)?' QUERY: MATCH (c1:candidato)-[:planeja]->(:plano)-[:contem]->(e1:eixo)-[:propoe]->(:ponto)-[s:similar_to]-(:ponto)<-[:propoe]-(:eixo)<-[:contem]-(:plano)<-[:planeja]-(c2:candidato) WHERE c1.nome = 'ROBERTO REQUIÃO DE MELLO E SILVA' AND c2.nome = 'LUCIANO DUCCI' WITH e1.STD_Header AS header, s.cos_similarity AS similarity, c1, c2 RETURN c1.nome AS nome_Luizao, c2.nome AS nome_comparação, header AS assunto, AVG(similarity) AS similaridade UNION MATCH (c1:candidato)-[:planeja]->(:plano)-[:contem]->(e1:eixo)-[:propoe]->(:ponto)-[s:similar_to]-(:ponto)<-[:propoe]-(:eixo)<-[:contem]-(:plano)<-[:planeja]-(c2:candidato) WHERE c1.nome = 'ROBERTO REQUIÃO DE MELLO E SILVA' AND c2.nome = 'LUIZ GOULARTE ALVES' WITH e1.STD_Header AS header, s.cos_similarity AS similarity, c1, c2 RETURN c1.nome AS nome_Luizao, c2.nome AS nome_comparação, header AS assunto, AVG(similarity) AS similaridade"
 ]

# Initialize the retriever
retrieverQ2 = Text2CypherRetriever(
    driver=driver,
    llm=llm,  # type: ignore
    neo4j_schema=schema,
    examples=examples,
)

def format_response(response):
    try: 
        to_return = {}
        to_return['CYPHER'] = response.metadata['cypher']
        contexto = ""
        for item in response.items:
            contexto = contexto + item.content + "\n"
        to_return["CONTEXTO"] = contexto
    except AttributeError:
        to_return = {"CYPHER": response['metadata']['cypher']}
        contexto = ""
        for item in response['items']:
            contexto = contexto + item['content'] + "\n"
        to_return["CONTEXTO"] = contexto
    return to_return
    
def Q2(pergunta):
    try: 
        response = retrieverQ2.search(query_text=pergunta)
    except:
        try:
             response = retrieverQ2.search(query_text=pergunta)
        except:
            try:
                response = retrieverQ2.search(query_text=pergunta)
            except:
                response = {'items': [{"content":"VAZIO"}], 'metadata': {'cypher':'ERRO na geração do "CYPHER'}}

    return format_response(response)

In [94]:
avaliacao(LISTA_PERGUNTAS, Q2)

PERGUNTA 1: Quais são os candidatos a eleição?
CYPHER: MATCH (c:candidato) RETURN c.nome AS Candidato ORDER BY Candidato
CONTEXTO: <Record Candidato='ANDREA DO ROCIO CALDAS'>
<Record Candidato='CRISTINA REIS GRAEML'>
<Record Candidato='EDUARDO PIMENTEL SLAVIERO'>
<Record Candidato='FELIPE GUSTAVO BOMBARDELLI'>
<Record Candidato='LUCIANO DUCCI'>
<Record Candidato='LUIZ GOULARTE ALVES'>
<Record Candidato='MARIA VICTORIA BORGHETTI BARROS'>
<Record Candidato='NEY LEPREVOST NETO'>
<Record Candidato='ROBERTO REQUIÃO DE MELLO E SILVA'>
<Record Candidato='SAMUEL DE MATTOS FIGUEIREDO'>

----------------------------------------------------------------------
PERGUNTA 2: Qual o partido do candidato Requião?
CYPHER: MATCH (c:candidato {nome: 'ROBERTO REQUIÃO DE MELLO E SILVA'}) RETURN c.partido AS Partido
CONTEXTO: <Record Partido='MOBILIZA'>

----------------------------------------------------------------------
PERGUNTA 3: Qual o partido de cada um dos candidatos?
CYPHER: MATCH (c:candidato) RETU

## 4.4 Chaining Manual usando text2cypher + LLM

In [74]:
from openai import OpenAI 

client = OpenAI(api_key=OPENAI_API_KEY)

def formatar_contexto(response):
    try:
        cypher = response.metadata['cypher']
        contexto = ""
        for item in response.items:
            contexto = contexto + item.content + "\n"
    except AttributeError:
        cypher = response['metadata']['cypher']
        contexto = ""
        for item in response['items']:
            contexto = contexto + item['content'] + "\n"
    return cypher, contexto

def Q3(pergunta, model="gpt-4o"):
   #Buscar cypher e contexto
   try: 
        response = retrieverQ2.search(query_text=pergunta)
   except:
        try:
             response = retrieverQ2.search(query_text=pergunta)
        except:
            try:
                response = retrieverQ2.search(query_text=pergunta)
            except:
                response = {'items': [{"content":"VAZIO"}], 'metadata': {'cypher':'ERRO na geração do "CYPHER'}}

   response_dic = {}
   response_dic['CYPHER'], response_dic['CONTEXTO'] =  formatar_contexto(response)
   
   system_prompt = "Você é uma assistente de perguntas e respostas que utiliza o contexto apresentado no prompt para responder à pergunta do usuário de forma precisa e eloquente." 
   prompt = f"""
Use apenas a informação apresentada neste contexto para respondes eloquentemente mas de maneira sucinta, a pergunta a seguir.
CONTEXTO:
{response_dic['CONTEXTO']}
PERGUNTA:
{pergunta}
"""
   completion = client.chat.completions.create(
        model=model, 
        temperature=0, # Não queremos criatividade, 
        messages=[
        {
            "role": "system",
            "content": system_prompt 
        },
        {
            "role": "user",
            "content": prompt
        }
        ]
    )

   response_dic['RESPOSTA'] = completion.choices[0].message.content

   return response_dic

In [95]:
avaliacao(LISTA_PERGUNTAS, Q3)

PERGUNTA 1: Quais são os candidatos a eleição?
CYPHER: MATCH (c:candidato) RETURN c.nome AS Candidato ORDER BY Candidato
CONTEXTO: <Record Candidato='ANDREA DO ROCIO CALDAS'>
<Record Candidato='CRISTINA REIS GRAEML'>
<Record Candidato='EDUARDO PIMENTEL SLAVIERO'>
<Record Candidato='FELIPE GUSTAVO BOMBARDELLI'>
<Record Candidato='LUCIANO DUCCI'>
<Record Candidato='LUIZ GOULARTE ALVES'>
<Record Candidato='MARIA VICTORIA BORGHETTI BARROS'>
<Record Candidato='NEY LEPREVOST NETO'>
<Record Candidato='ROBERTO REQUIÃO DE MELLO E SILVA'>
<Record Candidato='SAMUEL DE MATTOS FIGUEIREDO'>

RESPOSTA: Os candidatos à eleição são: Andrea do Rocio Caldas, Cristina Reis Graeml, Eduardo Pimentel Slaviero, Felipe
Gustavo Bombardelli, Luciano Ducci, Luiz Goularte Alves, Maria Victoria Borghetti Barros, Ney Leprevost Neto, Roberto
Requião de Mello e Silva, e Samuel de Mattos Figueiredo.
----------------------------------------------------------------------
PERGUNTA 2: Qual o partido do candidato Requião?
C

## 4.5 Langchain

In [104]:
from langchain.chains import GraphCypherQAChain
from langchain.prompts.prompt import PromptTemplate
from langchain_openai import ChatOpenAI

CYPHER_GENERATION_TEMPLATE = """Task:Generate Cypher statement to 
query a graph database.
Instructions:
Use only the provided relationship types and properties in the 
schema. Do not use any other relationship types or properties that 
are not provided.
Schema:
{schema}

Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than 
for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.
The node property ":eixo.STD_Header" will always be one of the headers contained on the lisT ["Introdução", "Educação", "Saúde", "Habitação e Ocupação Urbana", "Assistência e Ação Social", "Meio Ambiente e crise climática", "Segurança Pública", "Políticas Afirmativas e Direitos Humanos", "Cultura, Esporte, Lazer e Turismo", "Gestão e Transparência", "Segurança Alimentar", "Urbanismo", "Zeladoria e Serviços", "Transporte Público e Mobilidade", "Desenvolvimento Econômico, Emprego, Renda e Tecnologia", "Conclusão", "Funcionalismo Público", "Legislações Específicas / Outros]
The candidate names (:candidato.nome) to be used on the queries are: ["ANDREA DO ROCIO CALDAS", "EDUARDO PIMENTEL SLAVIERO", "MARIA VICTORIA BORGHETTI BARROS", "CRISTINA REIS GRAEML", "NEY LEPREVOST NETO", "LUCIANO DUCCI", "LUIZ GOULARTE ALVES", "ROBERTO REQUIÃO DE MELLO E SILVA", "SAMUEL DE MATTOS FIGUEIREDO", "FELIPE GUSTAVO BOMBARDELLI"]

Examples: Here are a few examples of generated Cypher 
statements for particular questions:

# Qual o nome dos candidatos a eleição?
MATCH (c:candidato) RETURN c.nome AS Candidato ORDER BY Candidato

# Qual o vice do candidato Ney Leprevost?
MATCH (c:candidato {{nome: 'NEY LEPREVOST NETO'}}) RETURN c.vice AS Vice

# Qual a coligação de cada um dos candidatos?
MATCH (c:candidato) RETURN c.nome AS Candidato, c.coalizao AS Coligação ORDER BY Candidato

# Quem são os Vices de cada candidato?
MATCH (c:candidato) RETURN c.nome AS Candidato, c.vice AS Vice ORDER BY Candidato

# Existem propostas específicas para o bairro Bacacheri?
WITH genai.vector.encode('Bacacheri', 'OpenAI', {{token:'sk-YOUR_KEY', model:'text-embedding-3-large'}}) AS query_vector CALL db.index.vector.queryNodes('ponto_vidx', 5000, query_vector) YIELD node, score WHERE score > 0.75 RETURN node.nome, node.texto, score ORDER BY score DESC

# Quais as propostas para a área da Educação?
MATCH (e:eixo {{STD_Header: 'Educação'}})-[:propoe]->(p:ponto) RETURN p.nome AS Proposta, p.texto AS Descrição ORDER BY p.nome

# Quais são as propostas para a saúde do candidato Maria Vitória?
MATCH (c:candidato {{nome: 'MARIA VICTORIA BORGHETTI BARROS'}})-[:planeja]->(:plano)-[:contem]->(e:eixo {{STD_Header: 'Saúde'}})-[:propoe]->(p:ponto) RETURN p.nome AS Proposta, p.texto AS Descrição ORDER BY Proposta

# Temos algum candidato que não inclui em seu plano o tema 
MATCH (c:candidato) WHERE NOT (c)-[:planeja]->(:plano)-[:contem]->(:eixo {{STD_Header: 'Zeladoria e Serviços'}})-[:propoe]->(:ponto) RETURN c.nome AS Candidato_que_não_contempla_o_tema ORDER BY Candidato_que_não_contempla_o_tema

# O Plano do candidato Andrea Caldas contempla quais tópicos?
MATCH (c:candidato)-[]-(:plano)-[]-(e:eixo) WHERE c.nome='ANDREA DO ROCIO CALDAS' RETURN e.Header_1

# Como o candidato Luizão Goulart enxerga a questão da educação básica?
WITH genai.vector.encode('educação básica', 'OpenAI', {{token:'sk-YOUR_KEY', model:'text-embedding-3-large'}}) AS query_vector CALL db.index.vector.queryNodes('ponto_vidx', 5000, query_vector) YIELD node, score WHERE score > 0.70 MATCH (c:candidato {{nome: 'LUIZ GOULARTE ALVES'}})-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(node) RETURN node.nome AS Proposta, node.texto AS Descrição, score ORDER BY  score DESC

# Como a candidata Cristina Graeml vê a questão da diversidade e gênero?
WITH genai.vector.encode('diversidade e gênero', 'OpenAI', {{token:'sk-YOUR_KEY', model:'text-embedding-3-large'}}) AS query_vector CALL db.index.vector.queryNodes('ponto_vidx', 5000, query_vector) YIELD node, score WHERE score > 0.70 MATCH (c:candidato {{nome: 'CRISTINA REIS GRAEML'}})-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(node) RETURN node.nome AS Proposta, node.texto AS Descrição, score ORDER BY  score DESC

# Que propostas o candidato Roberto Requião traz para o desenvovlimento turístico?
WITH genai.vector.encode('desenvovlimento turístico', 'OpenAI', {{token:'sk-YOUR_KEY', model:'text-embedding-3-large'}}) AS query_vector CALL db.index.vector.queryNodes('ponto_vidx', 5000, query_vector) YIELD node, score WHERE score > 0.70 MATCH (c:candidato {{nome: 'ROBERTO REQUIÃO DE MELLO E SILVA'}})-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(node) RETURN node.nome AS Proposta, node.texto AS Descrição, score ORDER BY  score DESC

# Qual candidato possui mais propostas na área da Habitação e Ocupação Urbana ?
MATCH (c:candidato)-[:planeja]->(:plano)-[:contem]->(e:eixo)-[:propoe]->(p:ponto) WHERE e.STD_Header = 'Habitação e Ocupação Urbana' WITH c.nome AS Candidato, COUNT(p) AS Numero_de_propostas_na_Saúde RETURN Candidato, Numero_de_propostas_na_Saúde ORDER BY Numero_de_propostas_na_Saúde DESC LIMIT 1

# Cada candidato tem um tema no qual foca maior quantidade de propostas. Quais são estes temas para de cada candidato?
MATCH (c:candidato)-[:planeja]->(:plano)-[:contem]->(e:eixo)-[:propoe]->(p:ponto) WITH c.nome AS candidato, e.STD_Header AS header, COUNT(p) AS total_propostas WITH candidato, header, total_propostas ORDER BY candidato, total_propostas DESC WITH candidato, COLLECT({{header: header, total: total_propostas}}) AS header_data RETURN candidato, header_data[0].header AS maior_header

# Qual dos candidatos a prefeito tem o menor número de propostas?
MATCH (c:candidato)-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(p:ponto) WITH c.nome AS Candidato, COUNT(p) AS Numero_de_propostas RETURN Candidato, Numero_de_propostas ORDER BY Numero_de_propostas LIMIT 1

# Qual candidato tem mais propostas na área da Assistência e Ação Social?
MATCH (c:candidato)-[:planeja]->(:plano)-[:contem]->(e:eixo {{STD_Header: 'Assistência e Ação Social'}})-[:propoe]->(p:ponto) WITH c.nome AS Candidato, COUNT(p) AS Numero_de_propostas_para_segurança_pública RETURN Candidato, Numero_de_propostas_para_segurança_pública ORDER BY Numero_de_propostas_para_segurança_pública DESC

# Em que áreas as propostas de Requião e de Eduardo Pimentel mais se assemelham e mais se diferenciam?
MATCH (c1:candidato)-[:planeja]->(:plano)-[:contem]->(e1:eixo)-[:propoe]->(p1:ponto)-[s:similar_to]-(p2:ponto)<-[:propoe]-(e2:eixo)<-[:contem]-(:plano)<-[:planeja]-(c2:candidato) WHERE c1.nome = 'ROBERTO REQUIÃO DE MELLO E SILVA' AND c2.nome = 'EDUARDO PIMENTEL SLAVIERO' WITH e1.STD_Header AS header, s.cos_similarity AS similarity RETURN header, AVG(similarity) AS avg_cos_similarity ORDER BY avg_cos_similarity DESC

# Qual candidato possui mais propostas para o modal de transporte bicicleta?
WITH genai.vector.encode('modal de transporte bicicleta', 'OpenAI', {{token:'sk-YOUR_KEY', model:'text-embedding-3-large'}}) AS query_vector CALL db.index.vector.queryNodes('ponto_vidx', 5000, query_vector) YIELD node, score WHERE score > 0.75 MATCH (c:candidato)-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(node) RETURN c.nome AS Candidato, COUNT(node) AS Numero_de_pontos ORDER BY Numero_de_pontos DESC

# Quais os candidatos que mais se assemelham entre si?
MATCH (c1:candidato)-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(p1:ponto)-[sim:similar_to]->(p2:ponto)<-[:propoe]-(:eixo)<-[:contem]-(:plano)<-[:planeja]-(c2:candidato) WHERE c1 <> c2 WITH c1.nome AS Candidato1, c2.nome AS Candidato2, AVG(sim.cos_similarity) AS Similarity RETURN Candidato1, Candidato2, Similarity ORDER BY Similarity DESC

# Meu candidato, o candidao Requião, não foi eleito para o segundo turno, estre os candidatos Luciano Ducci e Professor Luizão, quais têm propostas que melhor se assemelham às de meu candidato, por área de gestão (Saúde, Educação, etc)?
MATCH (c1:candidato)-[:planeja]->(:plano)-[:contem]->(e1:eixo)-[:propoe]->(:ponto)-[s:similar_to]-(:ponto)<-[:propoe]-(:eixo)<-[:contem]-(:plano)<-[:planeja]-(c2:candidato) WHERE c1.nome = 'ROBERTO REQUIÃO DE MELLO E SILVA' AND c2.nome = 'LUCIANO DUCCI' WITH e1.STD_Header AS header, s.cos_similarity AS similarity, c1, c2 RETURN c1.nome AS nome_Luizao, c2.nome AS nome_comparação, header AS assunto, AVG(similarity) AS similaridade UNION MATCH (c1:candidato)-[:planeja]->(:plano)-[:contem]->(e1:eixo)-[:propoe]->(:ponto)-[s:similar_to]-(:ponto)<-[:propoe]-(:eixo)<-[:contem]-(:plano)<-[:planeja]-(c2:candidato) WHERE c1.nome = 'ROBERTO REQUIÃO DE MELLO E SILVA' AND c2.nome = 'LUIZ GOULARTE ALVES' WITH e1.STD_Header AS header, s.cos_similarity AS similarity, c1, c2 RETURN c1.nome AS nome_Luizao, c2.nome AS nome_comparação, header AS assunto, AVG(similarity) AS similaridade

The question is:
{question}
"""
prompt = PromptTemplate.from_template(CYPHER_GENERATION_TEMPLATE)

cypherChain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0),
    graph=kg,
    allow_dangerous_requests=True,
    verbose=True,
    cypher_prompt=prompt,
    top_k=20
)

def Q4(question: str) -> str:
    prompt.format(schema=schema, question=question, OPENAI_API_KEY=OPENAI_API_KEY)
    try:
        response = cypherChain.run(question)
        print(textwrap.fill(response, 120))
    except Exception as e:
        print("########### ERRO ##########")
        print(e)

In [105]:
for pergunta in LISTA_PERGUNTAS:
    print("PERGUNTA:", pergunta)
    Q4(pergunta)
    print(40*"-")

PERGUNTA: Quais são os candidatos a eleição?


> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (c:candidato) RETURN c.nome AS Candidato ORDER BY Candidato
Full Context:
[{'Candidato': 'ANDREA DO ROCIO CALDAS'}, {'Candidato': 'CRISTINA REIS GRAEML'}, {'Candidato': 'EDUARDO PIMENTEL SLAVIERO'}, {'Candidato': 'FELIPE GUSTAVO BOMBARDELLI'}, {'Candidato': 'LUCIANO DUCCI'}, {'Candidato': 'LUIZ GOULARTE ALVES'}, {'Candidato': 'MARIA VICTORIA BORGHETTI BARROS'}, {'Candidato': 'NEY LEPREVOST NETO'}, {'Candidato': 'ROBERTO REQUIÃO DE MELLO E SILVA'}, {'Candidato': 'SAMUEL DE MATTOS FIGUEIREDO'}]

> Finished chain.
ANDREA DO ROCIO CALDAS, CRISTINA REIS GRAEML, EDUARDO PIMENTEL SLAVIERO, FELIPE GUSTAVO BOMBARDELLI, LUCIANO DUCCI, LUIZ
GOULARTE ALVES, MARIA VICTORIA BORGHETTI BARROS, NEY LEPREVOST NETO, ROBERTO REQUIÃO DE MELLO E SILVA, SAMUEL DE MATTOS
FIGUEIREDO.
----------------------------------------
PERGUNTA: Qual o partido do candidato Requião?


> Entering new GraphCypher

## 4.6 Neo4j Driver - Para aqueles queries mais difíceis

In [68]:
CYPHERS = [
"MATCH (c:candidato) RETURN c.nome AS Candidato ORDER BY Candidato",
"MATCH (c:candidato {nome: 'ROBERTO REQUIÃO DE MELLO E SILVA'}) RETURN c.partido AS Partido",
"MATCH (c:candidato) RETURN c.nome AS Candidato, c.partido AS Partido ORDER BY Candidato",
"MATCH (c:candidato) RETURN c.nome AS Candidato, c.vice AS Vice ORDER BY Candidato",
f"WITH genai.vector.encode('bairro Centro', 'OpenAI', {{token:'{OPENAI_API_KEY}', model:'text-embedding-3-large'}}) AS query_vector CALL db.index.vector.queryNodes('ponto_vidx', 5000, query_vector) YIELD node, score WHERE score > 0.75 RETURN node.nome, node.texto, score ORDER BY score DESC",
"MATCH (e:eixo {STD_Header: 'Saúde'})-[:propoe]->(p:ponto) RETURN p.nome AS Proposta, p.texto AS Descrição ORDER BY p.nome",
"MATCH (c:candidato {nome: 'ROBERTO REQUIÃO DE MELLO E SILVA'})-[:planeja]->(:plano)-[:contem]->(e:eixo {STD_Header: 'Saúde'})-[:propoe]->(p:ponto) RETURN p.nome AS Proposta, p.texto AS Descrição ORDER BY Proposta",
"MATCH (c:candidato) WHERE NOT (c)-[:planeja]->(:plano)-[:contem]->(:eixo {STD_Header: 'Políticas Afirmativas e Direitos Humanos'})-[:propoe]->(:ponto) RETURN c.nome AS Candidato_que_não_contempla_o_tema ORDER BY Candidato_que_não_contempla_o_tema",
"MATCH (c:candidato)-[]-(:plano)-[]-(e:eixo) WHERE c.nome='LUCIANO DUCCI' RETURN e.Header_1",
f"WITH genai.vector.encode('saúde pública', 'OpenAI', {{token:'{OPENAI_API_KEY}', model:'text-embedding-3-large'}}) AS query_vector CALL db.index.vector.queryNodes('ponto_vidx', 5000, query_vector) YIELD node, score WHERE score > 0.70 MATCH (c:candidato {{nome: 'LUIZ GOULARTE ALVES'}})-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(node) RETURN node.nome AS Proposta, node.texto AS Descrição, score ORDER BY  score DESC",
f"WITH genai.vector.encode('obrigatoriedade do plano de vacinação infantil', 'OpenAI', {{token:'{OPENAI_API_KEY}', model:'text-embedding-3-large'}}) AS query_vector CALL db.index.vector.queryNodes('ponto_vidx', 5000, query_vector) YIELD node, score WHERE score > 0.70 MATCH (c:candidato {{nome: 'CRISTINA REIS GRAEML'}})-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(node) RETURN node.nome AS Proposta, node.texto AS Descrição, score ORDER BY  score DESC",
f"WITH genai.vector.encode('ensino fundamental', 'OpenAI', {{token:'{OPENAI_API_KEY}', model:'text-embedding-3-large'}}) AS query_vector CALL db.index.vector.queryNodes('ponto_vidx', 5000, query_vector) YIELD node, score WHERE score > 0.70 MATCH (c:candidato {{nome: 'ROBERTO REQUIÃO DE MELLO E SILVA'}})-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(node) RETURN node.nome AS Proposta, node.texto AS Descrição, score ORDER BY  score DESC",
"MATCH (c:candidato)-[:planeja]->(:plano)-[:contem]->(e:eixo)-[:propoe]->(p:ponto) WHERE e.STD_Header = 'Saúde' WITH c.nome AS Candidato, COUNT(p) AS Numero_de_propostas_na_Saúde RETURN Candidato, Numero_de_propostas_na_Saúde ORDER BY Numero_de_propostas_na_Saúde DESC LIMIT 1",
"MATCH (c:candidato)-[:planeja]->(:plano)-[:contem]->(e:eixo)-[:propoe]->(p:ponto) WITH c.nome AS candidato, e.STD_Header AS header, COUNT(p) AS total_propostas WITH candidato, header, total_propostas ORDER BY candidato, total_propostas DESC WITH candidato, COLLECT({header: header, total: total_propostas}) AS header_data RETURN candidato, header_data[0].header AS maior_header",
"MATCH (c:candidato)-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(p:ponto) WITH c.nome AS Candidato, COUNT(p) AS Numero_de_propostas RETURN Candidato, Numero_de_propostas ORDER BY Numero_de_propostas DESC LIMIT 1",
"MATCH (c:candidato)-[:planeja]->(:plano)-[:contem]->(e:eixo {STD_Header: 'Segurança Pública'})-[:propoe]->(p:ponto) WITH c.nome AS Candidato, COUNT(p) AS Numero_de_propostas_para_segurança_pública RETURN Candidato, Numero_de_propostas_para_segurança_pública ORDER BY Numero_de_propostas_para_segurança_pública DESC",
"MATCH (c1:candidato)-[:planeja]->(:plano)-[:contem]->(e1:eixo)-[:propoe]->(p1:ponto)-[s:similar_to]-(p2:ponto)<-[:propoe]-(e2:eixo)<-[:contem]-(:plano)<-[:planeja]-(c2:candidato) WHERE c1.nome = 'LUCIANO DUCCI' AND c2.nome = 'NEY LEPREVOST NETO' WITH e1.STD_Header AS header, s.cos_similarity AS similarity RETURN header, AVG(similarity) AS avg_cos_similarity ORDER BY avg_cos_similarity DESC",
f"WITH genai.vector.encode('Transporte Coletivo', 'OpenAI', {{token:'{OPENAI_API_KEY}', model:'text-embedding-3-large'}}) AS query_vector CALL db.index.vector.queryNodes('ponto_vidx', 5000, query_vector) YIELD node, score WHERE score > 0.75 MATCH (c:candidato)-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(node) RETURN c.nome AS Candidato, COUNT(node) AS Numero_de_pontos ORDER BY Numero_de_pontos DESC",
"MATCH (c1:candidato)-[:planeja]->(:plano)-[:contem]->(:eixo)-[:propoe]->(p1:ponto)-[sim:similar_to]->(p2:ponto)<-[:propoe]-(:eixo)<-[:contem]-(:plano)<-[:planeja]-(c2:candidato) WHERE c1 <> c2 WITH c1.nome AS Candidato1, c2.nome AS Candidato2, AVG(sim.cos_similarity) AS Similarity RETURN Candidato1, Candidato2, Similarity ORDER BY Similarity DESC",
"MATCH (c1:candidato)-[:planeja]->(:plano)-[:contem]->(e1:eixo)-[:propoe]->(:ponto)-[s:similar_to]-(:ponto)<-[:propoe]-(:eixo)<-[:contem]-(:plano)<-[:planeja]-(c2:candidato) WHERE c1.nome = 'LUIZ GOULARTE ALVES' AND c2.nome = 'EDUARDO PIMENTEL SLAVIERO' WITH e1.STD_Header AS header, s.cos_similarity AS similarity, c1, c2 RETURN c1.nome AS nome_Luizao, c2.nome AS nome_comparação, header AS assunto, AVG(similarity) AS similaridade UNION MATCH (c1:candidato)-[:planeja]->(:plano)-[:contem]->(e1:eixo)-[:propoe]->(:ponto)-[s:similar_to]-(:ponto)<-[:propoe]-(:eixo)<-[:contem]-(:plano)<-[:planeja]-(c2:candidato) WHERE c1.nome = 'LUIZ GOULARTE ALVES' AND c2.nome = 'CRISTINA REIS GRAEML' WITH e1.STD_Header AS header, s.cos_similarity AS similarity, c1, c2 RETURN c1.nome AS nome_Luizao, c2.nome AS nome_comparação, header AS assunto, AVG(similarity) AS similaridade"
]

In [82]:
ex = ""
for per, cyp in zip(LISTA_PERGUNTAS, CYPHERS):
    ex = ex + "# " + per + "\n" + cyp + "\n\n"
ex = ex.replace("{", "{{")
ex = ex.replace("}", "}}")

In [42]:
from openai import OpenAI 
client = OpenAI(api_key=OPENAI_API_KEY)

def Q6(pergunta, cypher):
   response_dic = {}
   #Buscar contexto
   context = "\n ".join([str(r) for r in kg.query(cypher)])
   response_dic['CONTEXTO']=context
   system_prompt = "Você é uma assistente de perguntas e respostas que utiliza o contexto apresentado no prompt para responder à pergunta do usuário de forma precisa e eloquente." 
   prompt = f"""
Use apenas a informação apresentada neste contexto para respondes eloquentemente mas de maneira sucinta, a pergunta a seguir.
CONTEXTO:
{context}
PERGUNTA:
{pergunta}
"""
   completion = client.chat.completions.create(
        model="gpt-4o", 
        temperature=0, # Não queremos criatividade, 
        messages=[
        {
            "role": "system",
            "content": system_prompt 
        },
        {
            "role": "user",
            "content": prompt
        }
        ]
    )

   response_dic['RESPOSTA'] = completion.choices[0].message.content

   return response_dic

def avaliacaoQ6(perguntas,cyphers):
    i=1
    for pergunta, cypher in zip(perguntas, cyphers):
        print("PERGUNTA " + str(i) + ":", textwrap.fill(pergunta, 120))
        resposta = Q6(pergunta, cypher)
        f_resposta = ""
        for key in list(resposta.keys()):
            f_resposta = f_resposta + key + ": " + resposta[key] + "\n"
        print(text_formater(f_resposta))
        print(70*"-")
        i=i+1

In [69]:
avaliacaoQ6(LISTA_PERGUNTAS, CYPHERS)

PERGUNTA 1: Quais são os candidatos a eleição?
CONTEXTO: {'Candidato': 'ANDREA DO ROCIO CALDAS'}
 {'Candidato': 'CRISTINA REIS GRAEML'}
 {'Candidato': 'EDUARDO PIMENTEL SLAVIERO'}
 {'Candidato': 'FELIPE GUSTAVO BOMBARDELLI'}
 {'Candidato': 'LUCIANO DUCCI'}
 {'Candidato': 'LUIZ GOULARTE ALVES'}
 {'Candidato': 'MARIA VICTORIA BORGHETTI BARROS'}
 {'Candidato': 'NEY LEPREVOST NETO'}
 {'Candidato': 'ROBERTO REQUIÃO DE MELLO E SILVA'}
 {'Candidato': 'SAMUEL DE MATTOS FIGUEIREDO'}
RESPOSTA: Os candidatos à eleição são: Andrea do Rocio Caldas, Cristina Reis Graeml, Eduardo Pimentel Slaviero, Felipe
Gustavo Bombardelli, Luciano Ducci, Luiz Goularte Alves, Maria Victoria Borghetti Barros, Ney Leprevost Neto, Roberto
Requião de Mello e Silva, e Samuel de Mattos Figueiredo.
----------------------------------------------------------------------
PERGUNTA 2: Qual o partido do candidato Requião?
CONTEXTO: {'Partido': 'MOBILIZA'}
RESPOSTA: O partido do candidato Requião é o MOBILIZA.
------------------